In [1]:
pwd #working directory

'/home/wsuser/work'

In [2]:
!pip install keras==2.2.4

     |████████████████████████████████| 312 kB 21.9 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 15.9 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.2 requires keras<2.8,>=2.7.0, but you have keras 2.2.4 which is incompatible.


In [3]:
!pip install tensorflow

     |████████████████████████████████| 1.3 MB 21.4 MB/s eta 0:00:01
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4


In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1)

In [8]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='s6VPC2wtt5_qTUAJ83X3uZ6afZI5Out2YIDWLYK1klW0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'vegetablemodel-donotdelete-pr-i3cln2hzqydzyu'
object_key = 'veg-dataset.zip'

streaming_body_6 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [9]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_6.read()), "r")
file_paths = unzip.namelist() 
for path in file_paths:
    unzip.extract(path)

In [10]:
import os
filenames = os.listdir('/home/wsuser/work/veg-dataset')

In [12]:
x_train=train_datagen.flow_from_directory("/home/wsuser/work/veg-dataset/train_set",target_size=(128,128),class_mode='categorical',batch_size=32)

Found 11385 images belonging to 9 classes.


In [13]:
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/veg-dataset/test_set",target_size=(128,128),
                                        class_mode='categorical',batch_size=32)

Found 3416 images belonging to 9 classes.


In [14]:
x_train.class_indices

{'Pepper,_bell___Bacterial_spot': 0,
 'Pepper,_bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___Late_blight': 3,
 'Potato___healthy': 4,
 'Tomato___Bacterial_spot': 5,
 'Tomato___Late_blight': 6,
 'Tomato___Leaf_Mold': 7,
 'Tomato___Septoria_leaf_spot': 8}

In [15]:
model=Sequential() #load the model

In [16]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu')) #add convolutional layer

In [17]:
model.add(MaxPooling2D(pool_size=(2,2))) #add max pooling layer

In [18]:
model.add(Flatten()) #add flatten layer

In [22]:
model.add(Dense(units=128,kernel_initializer='uniform',activation='relu')) #add dense hidden layer

In [23]:
model.add(Dense(units=9,kernel_initializer='uniform',activation='softmax')) #add the output layer

In [24]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=["accuracy"]) #compile the model

In [25]:
model.fit(x_train,steps_per_epoch=356,epochs=5,validation_data=x_test,validation_steps=52)

Epoch 1/5
356/356 [==============================] - 238s 666ms/step - loss: 2.0964 - accuracy: 0.1828 - val_loss: 2.0925 - val_accuracy: 0.1881
Epoch 2/5
356/356 [==============================] - 238s 668ms/step - loss: 2.0746 - accuracy: 0.1838 - val_loss: 2.0856 - val_accuracy: 0.1911
Epoch 3/5
356/356 [==============================] - 236s 664ms/step - loss: 2.0739 - accuracy: 0.1868 - val_loss: 2.0959 - val_accuracy: 0.1989
Epoch 4/5
356/356 [==============================] - 236s 663ms/step - loss: 2.0749 - accuracy: 0.1868 - val_loss: 2.0830 - val_accuracy: 0.2049
Epoch 5/5
356/356 [==============================] - 244s 685ms/step - loss: 2.0744 - accuracy: 0.1868 - val_loss: 2.0880 - val_accuracy: 0.2067


In [26]:
model.save('vegdata.h5') #store model to use it locally

In [27]:
!tar -zcvf Train-model_new.tgz vegdata.h5 #convert to tgz file named as image-classification-model_new

vegdata.h5


In [28]:
#store model in repository

In [29]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 23.1 MB/s eta 0:00:01


In [30]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"Hdf86j0RNd5M-Y_NNeDjz9U9Xp2UC_cnj2Ij3QBADfeD" 
                  }

client = APIClient(wml_credentials)

In [31]:
client = APIClient(wml_credentials)

In [32]:
def guid_from_space_name(client, space_name): 
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [33]:
space_uid = guid_from_space_name(client, 'image_classification_vegetables') 
print("Space UID = " + space_uid)

Space UID = a190dcdc-9905-4fa3-b9ee-b62304d54f00


In [34]:
client.set.default_space(space_uid)

'SUCCESS'

In [35]:
client.software_specifications.list(limit=25)

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [36]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [37]:
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [38]:
import tensorflow
print(tensorflow.__version__)

2.7.2


In [39]:
ls

Train-model_new.tgz  vegdata.h5  veg-dataset/


In [40]:
model_details = client.repository.store_model(model= 'Train-model_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
    )

In [41]:
model_id = client.repository.get_model_id(model_details)

In [42]:
model_id

'6fac8993-8463-4305-a7a5-57427e1d10ba'

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 356)               45215204  
                                                                 
 dense_1 (Dense)             (None, 9)                 3213      
                                                                 
 dense_2 (Dense)             (None, 128)               1280      
                                                        

In [44]:
client.repository.download(model_id,'final_vegetable_model.tar.gz')

Successfully saved model content to file: 'final_vegetable_model.tar.gz'


'/home/wsuser/work/final_vegetable_model.tar.gz'

In [45]:
#test the model

In [46]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [47]:
model=load_model('vegdata.h5')

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='s6VPC2wtt5_qTUAJ83X3uZ6afZI5Out2YIDWLYK1klW0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'vegetablemodel-donotdelete-pr-i3cln2hzqydzyu'
object_key = 'b61b9d33-59ea-41b0-a554-c382eb6128c5___RS_HL 1919.JPG'

streaming_body_7 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

img= image.load_img(streaming_body_7,target_size=(64,64))

In [ ]:
x=image.img_to_array(img)

In [ ]:
x=np.expand_dims(x,axis=0)

In [ ]:
y=np.argmax(model.predict(x),axis=1)

In [ ]:
index=['Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot']

In [ ]:
index[y[0]]

In [50]:
!pip install numpy
!pip install flask
!pip install os
!pip install werkzeug

     |████████████████████████████████| 101 kB 14.1 MB/s ta 0:00:01
     |████████████████████████████████| 232 kB 62.5 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninstalled MarkupSafe-2.0.1
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [51]:
import requests
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import tensorflow as tf
from flask import Flask, request, render_template, redirect, url_for
import os
from werkzeug.utils import secure_filename

In [ ]:
app = Flask(__name__)

#load both the vegetable and fruit models
model = load_model("IBM-vegetable.h5")
model1=load_model("IBM-fruit.h5")

#home page
@app.route('/')
def home():
    return render_template('home.html')

#prediction page
@app.route('/prediction')
def prediction():
    return render_template('predict.html')

@app.route('/predict',methods=['POST'])		
def predict():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['image']

        # Save the file to ./uploads
        basepath = os.path.dirname(__file__)
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)
        img = image.load_img(file_path, target_size=(128, 128))
        
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        
        plant=request.form['plant']
        print(plant)
        if(plant=="vegetable"):
            preds = model.predict(x)
            preds=np.argmax(preds)
            print(preds)
            df=pd.read_excel('precautions - veg.xlsx')
            print(df.iloc[preds]['caution'])
        else:
            preds = model1.predict(x)
            preds=np.argmax(preds)                
            df=pd.read_excel('precautions - fruits.xlsx')
            print(df.iloc[preds]['caution'])
            
        
        return df.iloc[preds]['caution']
        
if __name__ == "__main__":
    app.run(debug=False)